In [ ]:
import time
import joblib
import openpyxl
import nbimporter
from tensorflow.keras.models import load_model

from feature_extraction import compute_bandpower, compute_stat_features

In [ ]:
sf = 128 # Hz
window_size = 10 # seconds
model = load_model('../models/mlp.keras')
scalar = joblib.load('../models/scalar_1d.pkl')

In [ ]:
streams = resolve_stream('type', 'EEG')
inlet = StreamInlet(streams[0])

In [ ]:
labels = []
start_time = time.time()

while time.time() - start_time <= 60
    row = []
    window = []
    window_start = time.time()

    while time.time() - window_start <= window_size:
        time.sleep(1/128)
        sample, _ = inlet.pull_sample()
        window.append(np.array(sample))

    window = np.array(window)

    for col in range(0, window.shape[1]):
        channel = window[:, col]
        theta = compute_bandpower(channel, 'theta')
        alpha = compute_bandpower(channel, 'alpha')
        beta = compute_bandpower(channel, 'beta')
        gamma = compute_bandpower(channel, 'gamma')

        beta_alpha_ratio = beta/alpha if alpha!=0 else 0

        mean, std, skewness, kurtosis, entropy = compute_stat_features(channel)

        row.extend([theta, alpha, beta, gamma, beta_alpha_ratio, mean, std, skewness, kurtosis, entropy])

    row = np.array(row).reshape(1, -1) # shape (1, 320)
    row = scalar.transform(row)
    prediction = model.predict(row)
    labels.append(int(np.round(prediction[0][0])))

focus_count = sum(labels)

if focus_count >= 4:
    output = 2

elif focus_count <= 2:
    output = 1

else:
    output = 0